In [1]:
from q_lab_toolbox.training_data import RandomTrainingData, mk_training_data
from q_lab_toolbox.target_systems import DecaySystem
from q_lab_toolbox.readout_operators import OrdernObservables

In [2]:
target = DecaySystem(ryd_interaction=0.1, 
                     omegas=(3.4, 0.2),
                     m=2,
                     gammas=(2, 1.2))

In [3]:
Os = OrdernObservables(2, 1)

In [4]:
data = RandomTrainingData(target_system=target,
                          N=3,
                          delta_T=0.1,
                          seed=42,
                          L=4,
                          Os=Os)

In [8]:
import numpy as np
ts, Ess, Os = mk_training_data(data)

In [10]:
np.save("test", ts)
np.save("test", Ess)
np.save("test", Os)